## 7. Group Assignment & Presentation



__You should be able to start up on this exercise after Lecture 1.__

*This exercise must be a group effort. That means everyone must participate in the assignment.*

In this assignment you will solve a data science problem end-to-end, pretending to be recently hired data scientists in a company. To help you get started, we've prepared a checklist to guide you through the project. Here are the main steps that you will go through:

1. Frame the problem and look at the big picture
2. Get the data
3. Explore and visualise the data to gain insights
4. Prepare the data to better expose the underlying data patterns to machine learning algorithms
5. Explore many different models and short-list the best ones
6. Fine-tune your models
7. Present your solution (video presentation) 

In each step we list a set of questions that one should have in mind when undertaking a data science project. The list is not meant to be exhaustive, but does contain a selection of the most important questions to ask. We will be available to provide assistance with each of the steps, and will allocate some part of each lesson towards working on the projects.

Your group must submit a _**single**_ Jupyter notebook, structured in terms of the first 6 sections listed above (the seventh will be a video uploaded to some streaming platform, e.g. YouTube, Vimeo, etc.).

### 1. Analysis: Frame the problem and look at the big picture
1. Find a problem/task that everyone in the group finds interesting
2. Define the objective in business terms
3. How should you frame the problem (supervised/unsupervised etc.)?
4. How should performance be measured?

### 2. Get the data
1. Find and document where you can get the data from
2. Get the data
3. Check the size and type of data (time series, geographical etc)

In [135]:
# Data found in https://github.com/dennis-ho/blackjack-simulator, with a small bug fix in the blackjack simulator where the dealer would bust even after player already busted.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('blackjack_simulator.csv')
data.head()



,shoe_id,cards_remaining,dealer_up,initial_hand,dealer_final,dealer_final_value,player_final,player_final_value,actions_taken,run_count,true_count,win
0,0,416,10,"[8, 4]","[10, 10]",20,"[[8, 4, 4, 4]]",[20],"[['H', 'H', 'S']]",1,0,0


### 3. Explore the data
1. Create a copy of the data for explorations (sampling it down to a manageable size if necessary)
2. Create a Jupyter notebook to keep a record of your data exploration
3. Study each feature and its characteristics:
    * Name
    * Type (categorical, int/float, bounded/unbounded, text, structured, etc)
    * Percentage of missing values
    * Check for outliers, rounding errors etc
4. For supervised learning tasks, identify the target(s)
5. Visualise the data
6. Study the correlations between features
7. Identify the promising transformations you may want to apply (e.g. convert skewed targets to normal via a log transformation)
8. Document what you have learned

In [136]:
import ast

# Only data we are interested on
data_filtered = data[["dealer_up", "initial_hand", "player_final", "player_final_value", "actions_taken"]]

# Use .loc to modify the DataFrame safely
data_filtered.loc[:, 'initial_hand'] = data_filtered['initial_hand'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
data_filtered.loc[:, 'player_final'] = data_filtered['player_final'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convert 'player_final_value' to lists of integers, handling 'BJ' as 21
data_filtered.loc[:, 'player_final_value'] = data_filtered['player_final_value'].apply(
    lambda x: [21 if i == 'BJ' else int(i) for i in ast.literal_eval(x)] if isinstance(x, str) else x
)

# Convert actions_taken to lists of strings
data_filtered.loc[:, 'actions_taken'] = data_filtered['actions_taken'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Check the types after conversion
print(data_filtered.dtypes)
print(data_filtered.head())

dealer_up              int64
initial_hand          object
player_final          object
player_final_value    object
actions_taken         object
dtype: object
   dealer_up initial_hand    player_final player_final_value actions_taken
0         10       [8, 4]  [[8, 4, 4, 4]]               [20]   [[H, H, S]]


In [137]:
# Initialize a new DataFrame to store the transformed data
expanded_rows = []

# Function to calculate the hand value with Ace as 11 if possible, otherwise as 1
def calculate_hand_value(hand, hit_or_double=False):
    total = sum(card if card != 11 else 1 for card in hand)  # Start with all Aces as 1
    aces = hand.count(11)  # Count the number of Aces
    while aces > 0 and total + 10 <= 21:
        total += 10  # Upgrade an Ace from 1 to 11
        aces -= 1
    
    # If "Hit" or "Double", check if total goes above 21 and downgrade Ace to 1
    if hit_or_double and total > 21:
        # Recalculate the value with all Aces counted as 1 and then upgrade them if possible
        total = sum(card if card != 11 else 1 for card in hand)  # Recalculate with Ace as 1
        aces = hand.count(11)  # Recalculate number of Aces
        while aces > 0 and total + 10 <= 21:
            total += 10  # Upgrade an Ace from 1 to 11
            aces -= 1

    return total

# Process each row in the DataFrame
for index, row in data_filtered.iterrows():
    hands = [row['initial_hand']]  # Start with the initial hand
    action_groups = row['actions_taken']  # Grouped actions per hand (after splits)

    # Process each hand and its corresponding actions
    for hand_idx, actions in enumerate(action_groups):
        current_hand = hands.pop(0)  # Take the current hand being processed
        for action_idx, action in enumerate(actions):
            # Calculate current hand value using the soft hand rule
            hit_or_double = action in ['H', 'D']
            hand_value = calculate_hand_value(current_hand, hit_or_double)

            # Ensure hand value doesn't exceed 21 after hit or double
            if hand_value > 21:
                hand_value = sum(card if card != 11 else 1 for card in current_hand)  # Recalculate with Ace as 1

            # Create a row for the current action
            new_row = {
                'dealer_up': row['dealer_up'],
                'current_hand': current_hand.copy(),  # State of the hand before action
                'action': action,
                'current_hand_value': hand_value,  # Always use the adjusted hand value
            }
            expanded_rows.append(new_row)

            # Handle split action (P)
            if action == 'P':
                # When splitting, split into two new hands
                split_hands = [[current_hand[0], row['player_final'][hand_idx][1]],
                               [current_hand[1], row['player_final'][hand_idx][0]]]
                hands.extend(split_hands)
                break  # Stop processing further actions for this hand since it split
            else:
                # Add a card to the current hand for actions like H or D
                if action in ['H', 'D']:
                    next_card = row['player_final'][hand_idx][len(current_hand)]
                    current_hand.append(next_card)

# Convert the expanded rows into a new DataFrame
expanded_data = pd.DataFrame(expanded_rows)

# Display the transformed DataFrame
expanded_data.to_csv("test.csv", index=False)


### 4. Prepare the data
Notes:
* Work on copies of the data (keep the original dataset intact).
* Write functions for all data transformations you apply, for three reasons:
    * So you can easily prepare the data the next time you run your code
    * So you can apply these transformations in future projects
    * To clean and prepare the test set
    
    
1. Data cleaning:
    * Fix or remove outliers (or keep them)
    * Fill in missing values (e.g. with zero, mean, median, regression ...) or drop their rows (or columns)
2. Feature selection (optional):
    * Drop the features that provide no useful information for the task (e.g. a customer ID is usually useless for modelling).
3. Feature engineering, where appropriate:
    * Discretize continuous features
    * Use one-hot encoding if/when relevant
    * Add promising transformations of features (e.g. $\log(x)$, $\sqrt{x}$, $x^2$, etc)
    * Aggregate features into promising new features
4. Feature scaling: standardise or normalise features

In [138]:
#calculate the bust probability for the dealers upcard
# odds based on https://www.cachecreek.com/blackjack-odds
def dealer_bust_probability(dealer_upcard):
    bust_probabilities = {
        2: 0.35,
        3: 0.37, 
        4: 0.40,  
        5: 0.42, 
        6: 0.42, 
        7: 0.26,  
        8: 0.24,  
        9: 0.23,  
        10: 0.10, 
        11: 0.10 
    }
    
    return bust_probabilities.get(dealer_upcard) 

#calculate the bust probability for the player
# odds based on https://www.cachecreek.com/blackjack-odds
def player_bust_probability(player_hand_total):
    # Bust probabilities based on the player's hand total
    bust_probabilities = {
        21: 1.00,  # if plays asks with 21 its 100% bust 
        20: 0.92,  
        19: 0.85, 
        18: 0.77, 
        17: 0.69, 
        16: 0.62,  
        15: 0.58, 
        14: 0.56,  
        13: 0.39, 
        12: 0.31   
    }
    
    return bust_probabilities.get(player_hand_total) 


expanded_data['dealer_bust_probability'] = expanded_data.apply(
    lambda row: dealer_bust_probability(int(row['dealer_up'])), axis=1
)
expanded_data['player_bust_probability'] = expanded_data.apply(
    lambda row: player_bust_probability(int(row['current_hand_value'])), axis=1
)
expanded_data.head()

,dealer_up,current_hand,action,current_hand_value,dealer_bust_probability,player_bust_probability
0,10,"[8, 4]",H,12,0.1,0.31
1,10,"[8, 4, 4]",H,16,0.1,0.62
2,10,"[8, 4, 4, 4]",S,20,0.1,0.92


### 5. Short-list promising models
We expect you to do some additional research and train at **least one model per team member**.

1. Train mainly quick and dirty models from different categories (e.g. linear, SVM, Random Forests etc) using default parameters
2. Measure and compare their performance
3. Analyse the most significant variables for each algorithm
4. Analyse the types of errors the models make
5. Have a quick round of feature selection and engineering if necessary
6. Have one or two more quick iterations of the five previous steps
7. Short-list the top three to five most promising models, preferring models that make different types of errors

### 6. Fine-tune the system
1. Fine-tune the hyperparameters
2. Once you are confident about your final model, measure its performance on the test set to estimate the generalisation error

### 7. Present your solution
1. Document what you have done
2. Create a nice 15 minute video presentation with slides
    * Make sure you highlight the big picture first
3. Explain why your solution achieves the business objective
4. Don't forget to present interesting points you noticed along the way:
    * Describe what worked and what did not
    * List your assumptions and you model's limitations
5. Ensure your key findings are communicated through nice visualisations or easy-to-remember statements (e.g. "the median income is the number-one predictor of housing prices")
6. Upload the presentation to some online platform, e.g. YouTube or Vimeo, and supply a link to the video in the notebook.

Géron, A. 2017, *Hands-On Machine Learning with Scikit-Learn and Tensorflow*, Appendix B, O'Reilly Media, Inc., Sebastopol.